In [24]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Add
import matplotlib.pyplot as plt
from IPython import display
from jiwer import wer
from tensorflow import argmax

In [25]:
data=pd.read_csv("total_data.csv")

# shuffle the rows using the sample() method
data = data.sample(frac=1).reset_index(drop=True)


split =int(len(data)*0.90)
df_train = data[:split]
df_val = data[split:]

In [26]:


print(f"Size of the training set: {len(df_train)}")
print(f"Size of the validation set: {len(df_val)}")

Size of the training set: 4003
Size of the validation set: 445


In [4]:
print(data.tail())

      Unnamed: 0               filename  \
4443         808  tef_04830_01749526999   
4444        1029  tem_00113_01019655999   
4445        1684  tef_04261_01067614539   
4446        1513  tem_05479_01926374514   
4447        2113  tem_03338_01085284958   

                                  transcription  \
4443          వాటి యొక్క పరిమాణం ఒక మిల్లీమీటరు   
4444      రెండు బాధలున్న వాడితో చెప్పుకున్నట్టు   
4445          మొదటి స్థానంలో జైపూర్ జిల్లా ఉంది   
4446  ఆమె ప్రసన్నురాలై యొక వ్యాకరణము నిచ్చెను\n   
4447         నా మాట వింటే సుయోధనుడు బాగు పడతాడు   

                                    filepath  
4443  te_in_female\tef_04830_01749526999.wav  
4444    end_to_end\tem_00113_01019655999.wav  
4445  te_in_female\tef_04261_01067614539.wav  
4446    end_to_end\tem_05479_01926374514.wav  
4447    end_to_end\tem_03338_01085284958.wav  


In [27]:
characters=[ 'అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ఎ', 'ఏ', 'ఐ', 'ఒ', 'ఓ', 'ఔ', 'క', 'ఖ', 'గ', 'ఘ', 'ఙ', 'చ', 'ఛ', 'జ', 'ఝ', 'ఞ', 'ట', 'ఠ', 'డ', 'ఢ', 'ణ', 'త', 'థ', 'ద', 'ధ', 'న', 'ప', 'ఫ', 'బ', 'భ', 'మ', 'య', 'ర', 'ఱ', 'ల', 
            'ళ', 'వ', 'శ', 'ష', 'స', 'హ','ం', 'ా', 'ి', 'ీ', 'ు', 'ూ', 'ృ', 'ె', 'ే', 'ై', 'ొ', 'ో', 'ౌ','్', ' '] 

In [28]:
#mapping chars to ints
char_to_num = keras.layers.StringLookup(vocabulary=characters,oov_token="")
#mapping ints backto chars
num_to_char = keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(),oov_token="",invert=True)
print(
    f"The vocabulary is: {char_to_num.get_vocabulary()}"
    f"(size={char_to_num.vocabulary_size()})"
)

The vocabulary is: ['', 'అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ఎ', 'ఏ', 'ఐ', 'ఒ', 'ఓ', 'ఔ', 'క', 'ఖ', 'గ', 'ఘ', 'ఙ', 'చ', 'ఛ', 'జ', 'ఝ', 'ఞ', 'ట', 'ఠ', 'డ', 'ఢ', 'ణ', 'త', 'థ', 'ద', 'ధ', 'న', 'ప', 'ఫ', 'బ', 'భ', 'మ', 'య', 'ర', 'ఱ', 'ల', 'ళ', 'వ', 'శ', 'ష', 'స', 'హ', 'ం', 'ా', 'ి', 'ీ', 'ు', 'ూ', 'ృ', 'ె', 'ే', 'ై', 'ొ', 'ో', 'ౌ', '్', ' '](size=64)


In [29]:
import librosa
import os
from tensorflow_io import audio as tfio

# the window length in samples
frame_length = 256
# the number of samples to step
frame_step = 160
# the number of FFT points
n_fft = 384
# the number of Mel frequency bins
n_mels = 128

def encode_single_sample(wav_file, label, path):
  # read the WAV file
  file = tf.io.read_file(path)
  # decode the WAV file
  audio = tfio.decode_wav(file, shape=None, dtype=tf.int16)
  audio = tf.squeeze(audio, axis=-1)
  audio=tf.cast(audio,tf.float32)
  # compute the Mel spectrogram
  spectrogram = tf.signal.stft(audio, frame_length=frame_length, frame_step=frame_step, fft_length=n_fft, window_fn=tf.signal.hann_window)
  spectrogram = tf.abs(spectrogram)
  spectrogram = tf.square(spectrogram)
  mel_filterbank = tf.signal.linear_to_mel_weight_matrix(num_mel_bins=n_mels, num_spectrogram_bins=n_fft//2+1, sample_rate=16000, lower_edge_hertz=0.0, upper_edge_hertz=8000.0)
  mel_spectrogram = tf.matmul(spectrogram, mel_filterbank)
  mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)
  # normalisation
  means = tf.math.reduce_mean(mel_spectrogram, axis=1, keepdims=True)
  stddevs = tf.math.reduce_std(mel_spectrogram, axis=1, keepdims=True)
  mel_spectrogram = (mel_spectrogram - means) / (stddevs + 1e-10)
  # convert label to lower case
  label = tf.strings.lower(label)
  # split the label
  label = tf.strings.unicode_split(label, input_encoding="UTF-8")
  # map the characters in labels to numbers
  label = char_to_num(label)
  # return parts
  return mel_spectrogram, label

In [30]:
batch_size = 3
#define the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_train["filename"]),list(df_train["transcription"]),list(df_train["filepath"]))
)
train_dataset = (
    train_dataset.map(encode_single_sample,num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)


#define the validation dataset
validation_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_val["filename"]),list(df_val["transcription"]),list(df_val["filepath"]))
)
validation_dataset=(
    validation_dataset.map(encode_single_sample,num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

In [31]:
for input_spectrograms, target_transcripts in train_dataset:
    print("Input spectrograms shape:", input_spectrograms.shape)
    print("Target transcripts shape:", target_transcripts.shape)

Input spectrograms shape: (3, 1996, 128)
Target transcripts shape: (3, 42)
Input spectrograms shape: (3, 1817, 128)
Target transcripts shape: (3, 38)
Input spectrograms shape: (3, 2559, 128)
Target transcripts shape: (3, 32)
Input spectrograms shape: (3, 1689, 128)
Target transcripts shape: (3, 32)
Input spectrograms shape: (3, 1765, 128)
Target transcripts shape: (3, 42)
Input spectrograms shape: (3, 1689, 128)
Target transcripts shape: (3, 50)
Input spectrograms shape: (3, 1714, 128)
Target transcripts shape: (3, 54)
Input spectrograms shape: (3, 1253, 128)
Target transcripts shape: (3, 37)
Input spectrograms shape: (3, 1586, 128)
Target transcripts shape: (3, 36)
Input spectrograms shape: (3, 1253, 128)
Target transcripts shape: (3, 46)
Input spectrograms shape: (3, 1919, 128)
Target transcripts shape: (3, 40)
Input spectrograms shape: (3, 2738, 128)
Target transcripts shape: (3, 38)
Input spectrograms shape: (3, 2149, 128)
Target transcripts shape: (3, 41)
Input spectrograms shape:

Input spectrograms shape: (3, 1663, 128)
Target transcripts shape: (3, 46)
Input spectrograms shape: (3, 1868, 128)
Target transcripts shape: (3, 67)
Input spectrograms shape: (3, 1612, 128)
Target transcripts shape: (3, 36)
Input spectrograms shape: (3, 1996, 128)
Target transcripts shape: (3, 40)
Input spectrograms shape: (3, 2303, 128)
Target transcripts shape: (3, 33)
Input spectrograms shape: (3, 1202, 128)
Target transcripts shape: (3, 48)
Input spectrograms shape: (3, 1484, 128)
Target transcripts shape: (3, 34)
Input spectrograms shape: (3, 1945, 128)
Target transcripts shape: (3, 67)
Input spectrograms shape: (3, 2149, 128)
Target transcripts shape: (3, 36)
Input spectrograms shape: (3, 1509, 128)
Target transcripts shape: (3, 58)
Input spectrograms shape: (3, 1893, 128)
Target transcripts shape: (3, 56)
Input spectrograms shape: (3, 1330, 128)
Target transcripts shape: (3, 39)
Input spectrograms shape: (3, 1228, 128)
Target transcripts shape: (3, 40)
Input spectrograms shape:

Input spectrograms shape: (3, 3429, 128)
Target transcripts shape: (3, 64)
Input spectrograms shape: (3, 2738, 128)
Target transcripts shape: (3, 38)
Input spectrograms shape: (3, 1893, 128)
Target transcripts shape: (3, 37)
Input spectrograms shape: (3, 1484, 128)
Target transcripts shape: (3, 37)
Input spectrograms shape: (3, 1253, 128)
Target transcripts shape: (3, 31)
Input spectrograms shape: (3, 1663, 128)
Target transcripts shape: (3, 38)
Input spectrograms shape: (3, 1356, 128)
Target transcripts shape: (3, 51)
Input spectrograms shape: (3, 3071, 128)
Target transcripts shape: (3, 69)
Input spectrograms shape: (3, 1893, 128)
Target transcripts shape: (3, 32)
Input spectrograms shape: (3, 1202, 128)
Target transcripts shape: (3, 59)
Input spectrograms shape: (3, 2380, 128)
Target transcripts shape: (3, 49)
Input spectrograms shape: (3, 1509, 128)
Target transcripts shape: (3, 42)
Input spectrograms shape: (3, 1509, 128)
Target transcripts shape: (3, 39)
Input spectrograms shape:

Input spectrograms shape: (3, 2354, 128)
Target transcripts shape: (3, 69)
Input spectrograms shape: (3, 1868, 128)
Target transcripts shape: (3, 65)
Input spectrograms shape: (3, 1817, 128)
Target transcripts shape: (3, 39)
Input spectrograms shape: (3, 1151, 128)
Target transcripts shape: (3, 54)
Input spectrograms shape: (3, 2149, 128)
Target transcripts shape: (3, 50)
Input spectrograms shape: (3, 1765, 128)
Target transcripts shape: (3, 42)
Input spectrograms shape: (3, 1407, 128)
Target transcripts shape: (3, 45)
Input spectrograms shape: (3, 1612, 128)
Target transcripts shape: (3, 40)
Input spectrograms shape: (3, 2047, 128)
Target transcripts shape: (3, 53)
Input spectrograms shape: (3, 2815, 128)
Target transcripts shape: (3, 88)
Input spectrograms shape: (3, 1458, 128)
Target transcripts shape: (3, 40)
Input spectrograms shape: (3, 2815, 128)
Target transcripts shape: (3, 62)
Input spectrograms shape: (3, 1740, 128)
Target transcripts shape: (3, 56)
Input spectrograms shape:

Input spectrograms shape: (3, 1407, 128)
Target transcripts shape: (3, 36)
Input spectrograms shape: (3, 1945, 128)
Target transcripts shape: (3, 44)
Input spectrograms shape: (3, 1228, 128)
Target transcripts shape: (3, 30)
Input spectrograms shape: (3, 1458, 128)
Target transcripts shape: (3, 37)
Input spectrograms shape: (3, 1714, 128)
Target transcripts shape: (3, 40)
Input spectrograms shape: (3, 2892, 128)
Target transcripts shape: (3, 65)
Input spectrograms shape: (3, 2533, 128)
Target transcripts shape: (3, 80)
Input spectrograms shape: (3, 2098, 128)
Target transcripts shape: (3, 68)
Input spectrograms shape: (3, 1765, 128)
Target transcripts shape: (3, 42)
Input spectrograms shape: (3, 2533, 128)
Target transcripts shape: (3, 52)
Input spectrograms shape: (3, 3122, 128)
Target transcripts shape: (3, 60)
Input spectrograms shape: (3, 1612, 128)
Target transcripts shape: (3, 52)
Input spectrograms shape: (3, 1817, 128)
Target transcripts shape: (3, 40)
Input spectrograms shape:

...

Input spectrograms shape: (3, 1817, 128)
Target transcripts shape: (3, 39)
Input spectrograms shape: (3, 2098, 128)
Target transcripts shape: (3, 41)
Input spectrograms shape: (3, 2764, 128)
Target transcripts shape: (3, 39)
Input spectrograms shape: (3, 1586, 128)
Target transcripts shape: (3, 39)
Input spectrograms shape: (3, 1842, 128)
Target transcripts shape: (3, 39)
Input spectrograms shape: (3, 2482, 128)
Target transcripts shape: (3, 60)
Input spectrograms shape: (3, 1842, 128)
Target transcripts shape: (3, 39)
Input spectrograms shape: (3, 1791, 128)
Target transcripts shape: (3, 38)
Input spectrograms shape: (3, 2226, 128)
Target transcripts shape: (3, 36)
Input spectrograms shape: (3, 1228, 128)
Target transcripts shape: (3, 40)
Input spectrograms shape: (3, 2431, 128)
Target transcripts shape: (3, 59)
Input spectrograms shape: (3, 1765, 128)
Target transcripts shape: (3, 35)
Input spectrograms shape: (3, 3071, 128)
Target transcripts shape: (3, 34)
Input spectrograms shape:

Input spectrograms shape: (3, 3071, 128)
Target transcripts shape: (3, 85)
Input spectrograms shape: (3, 2021, 128)
Target transcripts shape: (3, 62)
Input spectrograms shape: (3, 2585, 128)
Target transcripts shape: (3, 79)
Input spectrograms shape: (3, 1919, 128)
Target transcripts shape: (3, 34)
Input spectrograms shape: (3, 1586, 128)
Target transcripts shape: (3, 81)
Input spectrograms shape: (3, 2098, 128)
Target transcripts shape: (3, 40)
Input spectrograms shape: (3, 2175, 128)
Target transcripts shape: (3, 52)
Input spectrograms shape: (3, 1074, 128)
Target transcripts shape: (3, 33)
Input spectrograms shape: (3, 1842, 128)
Target transcripts shape: (3, 50)
Input spectrograms shape: (3, 1663, 128)
Target transcripts shape: (3, 48)
Input spectrograms shape: (3, 1689, 128)
Target transcripts shape: (3, 49)
Input spectrograms shape: (3, 2201, 128)
Target transcripts shape: (3, 54)
Input spectrograms shape: (3, 2380, 128)
Target transcripts shape: (3, 76)
Input spectrograms shape:

Input spectrograms shape: (3, 1484, 128)
Target transcripts shape: (3, 24)
Input spectrograms shape: (3, 2098, 128)
Target transcripts shape: (3, 48)
Input spectrograms shape: (3, 2226, 128)
Target transcripts shape: (3, 53)
Input spectrograms shape: (3, 1279, 128)
Target transcripts shape: (3, 44)
Input spectrograms shape: (3, 1228, 128)
Target transcripts shape: (3, 37)
Input spectrograms shape: (3, 1458, 128)
Target transcripts shape: (3, 36)
Input spectrograms shape: (3, 1279, 128)
Target transcripts shape: (3, 44)
Input spectrograms shape: (3, 1509, 128)
Target transcripts shape: (3, 38)
Input spectrograms shape: (3, 1765, 128)
Target transcripts shape: (3, 68)
Input spectrograms shape: (3, 1151, 128)
Target transcripts shape: (3, 32)
Input spectrograms shape: (3, 2457, 128)
Target transcripts shape: (3, 57)
Input spectrograms shape: (3, 2047, 128)
Target transcripts shape: (3, 48)
Input spectrograms shape: (3, 1151, 128)
Target transcripts shape: (3, 37)
Input spectrograms shape:

Input spectrograms shape: (3, 1765, 128)
Target transcripts shape: (3, 54)
Input spectrograms shape: (3, 2047, 128)
Target transcripts shape: (3, 36)
Input spectrograms shape: (3, 1842, 128)
Target transcripts shape: (3, 38)
Input spectrograms shape: (3, 1663, 128)
Target transcripts shape: (3, 39)
Input spectrograms shape: (3, 1842, 128)
Target transcripts shape: (3, 57)
Input spectrograms shape: (3, 2124, 128)
Target transcripts shape: (3, 59)
Input spectrograms shape: (3, 2610, 128)
Target transcripts shape: (3, 60)
Input spectrograms shape: (3, 1740, 128)
Target transcripts shape: (3, 54)
Input spectrograms shape: (3, 1202, 128)
Target transcripts shape: (3, 34)
Input spectrograms shape: (3, 2047, 128)
Target transcripts shape: (3, 65)
Input spectrograms shape: (3, 2303, 128)
Target transcripts shape: (3, 50)
Input spectrograms shape: (3, 1637, 128)
Target transcripts shape: (3, 34)
Input spectrograms shape: (3, 1817, 128)
Target transcripts shape: (3, 56)
Input spectrograms shape:

Input spectrograms shape: (3, 1765, 128)
Target transcripts shape: (3, 45)
Input spectrograms shape: (3, 1586, 128)
Target transcripts shape: (3, 35)
Input spectrograms shape: (3, 1996, 128)
Target transcripts shape: (3, 36)
Input spectrograms shape: (3, 1228, 128)
Target transcripts shape: (3, 47)
Input spectrograms shape: (3, 1919, 128)
Target transcripts shape: (3, 74)
Input spectrograms shape: (3, 2713, 128)
Target transcripts shape: (3, 74)
Input spectrograms shape: (3, 1868, 128)
Target transcripts shape: (3, 57)
Input spectrograms shape: (3, 2713, 128)
Target transcripts shape: (3, 68)
Input spectrograms shape: (3, 1842, 128)
Target transcripts shape: (3, 25)
Input spectrograms shape: (3, 1023, 128)
Target transcripts shape: (3, 31)
Input spectrograms shape: (3, 2175, 128)
Target transcripts shape: (3, 92)
Input spectrograms shape: (3, 1484, 128)
Target transcripts shape: (3, 41)
Input spectrograms shape: (3, 1868, 128)
Target transcripts shape: (3, 40)
Input spectrograms shape:

In [32]:
import numpy as np
from numpy import reshape
def CTCLoss(y_true,y_pred):



  #compute training time loss value
  #y_prye=tf.argmax(y_pred,axis=2)
  batch_len = tf.cast(tf.shape(y_true)[0],dtype="int64")
  input_length = tf.cast(tf.shape(y_pred)[1],dtype="int64")
  label_length = tf.cast(tf.shape(y_true)[1],dtype="int64")
  input_length = input_length * tf.ones(shape=(batch_len,1),dtype="int64")
  label_length = label_length * tf.ones(shape=(batch_len,1),dtype="int64")
  loss = keras.backend.ctc_batch_cost(y_true,y_pred,input_length,label_length)
  return loss

def build_model(input_dim,output_dim):
  #Models input
  input_spectrogram = layers.Input((None,input_dim),name="input")
  #Expand the dimension to use 2D CNN
  x = layers.Reshape((-1, input_dim, 1),name="expand_dim")(input_spectrogram)
  #convolution layer 1
  x = layers.Conv2D(
      filters=32,
      kernel_size=[3,3],
      strides=[2,2],
      padding="same",
      use_bias=False,
      name="conv_1",
  )(x)
  x = layers.BatchNormalization(name="conv_1_bn")(x)
  x = layers.ReLU(name="conv_1_relu")(x)
  #convolution layer 2
  x = layers.Conv2D(
      filters=32,
      kernel_size=[3,3],
      strides=[1,2],
      padding="same",
      use_bias=False,
      name="conv_2",
  )(x)
  x = layers.BatchNormalization(name="conv_2_bn")(x)
  x = layers.ReLU(name="conv_2_relu")(x)
  x = layers.Reshape((-1,x.shape[-2] * x.shape[-1]))(x)
  # Add an LSTM layer
  x = layers.LSTM(64, return_sequences=True, name="lstm")(x)


   # classification layer
  output = layers.TimeDistributed(layers.Dense(units=output_dim + 1,activation="softmax"))(x)
  
   #Model
  model=keras.Model(input_spectrogram,output,name="TARUS")
   #Optimizer
  opt = keras.optimizers.Adam(learning_rate=1e-2)
   #compile the model and return 
  model.compile(optimizer=opt,loss=CTCLoss)
  return model

# get the model
model = build_model(
    input_dim=128,
    output_dim=char_to_num.vocabulary_size(),
)
#model.Add(argmax_layer())
model.summary(line_length=110)

Model: "TARUS"
______________________________________________________________________________________________________________
 Layer (type)                                    Output Shape                                Param #          
 input (InputLayer)                              [(None, None, 128)]                         0                
                                                                                                              
 expand_dim (Reshape)                            (None, None, 128, 1)                        0                
                                                                                                              
 conv_1 (Conv2D)                                 (None, None, 64, 32)                        288              
                                                                                                              
 conv_1_bn (BatchNormalization)                  (None, None, 64, 32)                        128 

In [33]:
model.load_weights('testing_weights2222.h5',by_name=True,skip_mismatch=True)

In [34]:
import Levenshtein

def calculate_cer_batch(predicted_batch, target_batch):
    """
    Calculate the average Character Error Rate (CER) for a batch of predicted and target texts.

    Args:
        predicted_batch (list of str): The predicted texts.
        target_batch (list of str): The target (ground truth) texts.

    Returns:
        float: The average CER.
    """
    total_cer = 0.0
    for i in range(len(predicted_batch)):
        predicted_text = predicted_batch[i].replace(" ", "")
        target_text = target_batch[i].replace(" ", "")

        distance = Levenshtein.distance(predicted_text, target_text)

        cer = distance / max(len(predicted_text), len(target_text))
        total_cer += cer

    avg_cer = total_cer / len(predicted_batch)
    return avg_cer


In [35]:
#Training and Evaluating

# A utility function to decode the output of the network
def decode_batch_predictions(pred):
  input_len = np.ones(pred.shape[0]) * pred.shape[1]
  #use greedy search. for coplex tasks, you can use beam search
  results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=False)[0][0]
  #Itrate over the results and get back the text
  output_text = []
  for result in results:
    result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
    output_text.append(result)
  return output_text

# A callback class to output a few transcriptions during training
class CallbackEval(keras.callbacks.Callback):
  """Display abatch of outputs after every epoch."""

  def __init__(self, dataset):
    super().__init__()
    self.dataset = dataset

  def on_epoch_end(self, epoch: int, logs=None):
    predictions = []
    targets = []
    for batch in self.dataset:
      x, y = batch
      batch_predictions = model.predict(x)
      batch_predictions = decode_batch_predictions(batch_predictions)
      predictions.extend(batch_predictions)
      for label in y:
        label = (
            tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        )
        targets.append(label)
    #wer_score = wer(targets, predictions)
    avg_cer = calculate_cer_batch(predictions, targets)
    print("-"* 100)
    print("Average CER: {:.2%}".format(avg_cer))
    
    #print(f"Word Error Rate: {wer_score:.4f}")
    print("-" * 100)
    for i in np.random.randint(0, len(predictions), 5):

      print(f"Target    : {targets[i]}")
      print(f"Prediction: {predictions[i]}")
      print("-" * 100)

In [36]:
#Lets start the training process.
for i in range(0,10):
    
    #Define the number of epochs
    epochs = 5
    #callback function to check transcription on the val set
    validation_callback = CallbackEval(validation_dataset)
    #Train the model

    history = model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=epochs,
        callbacks=[validation_callback],
    )
    model.save_weights('testing_weights_lstm_3.h5')

Epoch 1/5
1/1 [==============================] - 0s 45ms/step
----------------------------------------------------------------------------------------------------
Average CER: 93.50%
----------------------------------------------------------------------------------------------------
Target    : ఉదా స రి గ మ ప మ ప ద ని సా
Prediction: 
----------------------------------------------------------------------------------------------------
Target    : రవీంద్రుని రచనలలో గీతాంజలి చాల గొప్పది
Prediction: నలుి
----------------------------------------------------------------------------------------------------
Target    : బషీరాబాద్ రాజకీయ చైతన్యంగా ఎదిగిన గ్రామం
Prediction:  తిు
----------------------------------------------------------------------------------------------------
Target    : ఇందుకు ఉపయోగపడే ప్రచార వ్యాసం ఇక్కడ ఉన్నది
Prediction:  కు
----------------------------------------------------------------------------------------------------
Target    : క్రమంగా సంపూర్ణంగా అంతరించింది
Predicti

Epoch 2/5
1/1 [==============================] - 0s 45ms/step
----------------------------------------------------------------------------------------------------
Average CER: 66.99%
----------------------------------------------------------------------------------------------------
Target    : ఇలా పాటలు పూర్తిగా వ్రాయడం కాపీ హక్కుల ఉల్లంఘన కిందికి రావచ్చును
Prediction: లపతలపతిరడంకలోలంిదిర 
----------------------------------------------------------------------------------------------------
Target    : అన్నింటినీ మించి రససిద్ధి పరిమళిస్తుంది
Prediction: అనింమటినినికర సతికరమలసది
----------------------------------------------------------------------------------------------------
Target    : ఎం బీ ఏ కూడా అక్కడే చదివాడు
Prediction: మనకరమకరచమట
----------------------------------------------------------------------------------------------------
Target    : మీరు మీ గమ్యస్థానాన్ని చేరుకోబోతున్నారు
Prediction: మరమగుమసతనకచిరుకవతనరు
------------------------------------------------------------------

Epoch 3/5
1/1 [==============================] - 0s 41ms/step
----------------------------------------------------------------------------------------------------
Average CER: 59.26%
----------------------------------------------------------------------------------------------------
Target    : క్రమంగా తూర్పు చాళుక్యుల రాజ్యం దక్షిణాన నెల్లూరు నుండి ఉత్తరాన శ్రీకాకుళం వరకు విస్తరించింది
Prediction: కమంనతురపుచలకిలరచయదకనలురనండితరనచకలమరకసపరంచింది
----------------------------------------------------------------------------------------------------
Target    : ఆ బొమ్మను నొక్కి అసలు సైజు బొమ్మను చూడవచ్చు కూడా
Prediction: కుమంన సలుసజుబమనిచుడుచుకు
----------------------------------------------------------------------------------------------------
Target    : అంపైర్లే పాయింట్లను నిర్ణయిస్తారు
Prediction: ంపిలిపంనిస్రు
----------------------------------------------------------------------------------------------------
Target    : రుచి కూడా వికారం పుట్తించునట్లుండును
Prediction: ిచకుడికరం్ుటించినక

Epoch 4/5
1/1 [==============================] - 0s 57ms/step
----------------------------------------------------------------------------------------------------
Average CER: 56.60%
----------------------------------------------------------------------------------------------------
Target    : ఈ వివాదం సెప్టెంబర్ రెండు వెలు తొమ్మిదిన పరిష్కారమైనది
Prediction: వదంస్కటభరిడుతమిదలుపరి్కరమది
----------------------------------------------------------------------------------------------------
Target    : ఉదాహరణకుగైడ్ టు మాస్టర్బేషన్ మొదలైనవి
Prediction: ుచనరనకగటుమచ్ర్ంద్లన
----------------------------------------------------------------------------------------------------
Target    : అమెరికన్ ఆస్టియోపాథిక్ అసోసియేషన్
Prediction: మకిసతిపతి్సషచ
----------------------------------------------------------------------------------------------------
Target    : అప్పుడు మీ సంతకం ఇలా కనిపిస్తుంది
Prediction: పఅపుడుమనిసందటకంనిలకనిదలస్తంది
----------------------------------------------------------------

Epoch 5/5
1/1 [==============================] - 0s 49ms/step
----------------------------------------------------------------------------------------------------
Average CER: 55.31%
----------------------------------------------------------------------------------------------------
Target    : ఇక్కడ ఇప్పటికే శ్రీకాకుళం పైన ఒక వ్యాసం ఉంది
Prediction: ఇికడిపడికిక కలంపైన కవియసమంది
----------------------------------------------------------------------------------------------------
Target    : మాచ్ పూర్తిగా అమ్ముడు పోయింది
Prediction: చ్పు్తి మడుపంది
----------------------------------------------------------------------------------------------------
Target    : నాడీ కణాలు జ్ఞాన కణాలు లేవు
Prediction: నడికనమలునకనంలి
----------------------------------------------------------------------------------------------------
Target    : పొదయూకలిప్టస్ గ్లాబులస్
Prediction: వ కలటచదలమలచ
----------------------------------------------------------------------------------------------------
Target    : నీవు 

...

Epoch 1/5
1/1 [==============================] - 0s 69ms/step
----------------------------------------------------------------------------------------------------
Average CER: 79.94%
----------------------------------------------------------------------------------------------------
Target    : ఐఎండిబిలో శివరామకృష్ణయ్య వివరాలు
Prediction: అడిదువల
----------------------------------------------------------------------------------------------------
Target    : అయోధ్యను సాకేతపురమని కూడా అంటారు
Prediction: అచతంచ్కరనకుటర
----------------------------------------------------------------------------------------------------
Target    : భారతీయ తపాలా వ్యవస్థలో అతి ముఖ్యమైన విభాగము
Prediction: తపలశతలదనమన
----------------------------------------------------------------------------------------------------
Target    : ఎక్కువమంది స్థానికులు హిందువులు
Prediction: కునసతనలదుల
----------------------------------------------------------------------------------------------------
Target    : మండల పరిషత్తు ప్రా

Epoch 2/5
1/1 [==============================] - 0s 62ms/step
----------------------------------------------------------------------------------------------------
Average CER: 76.86%
----------------------------------------------------------------------------------------------------
Target    : ఆకులు గుండ్రముగా ఐదు తమ్మెలేర్పడియుండును
Prediction: అనమండ తండలరడింను
----------------------------------------------------------------------------------------------------
Target    : ఏమయినా అయోమయ నివృత్తి పేజీలను తయారు చేద్దాం
Prediction: నమనఅనికత్ిలస
----------------------------------------------------------------------------------------------------
Target    : ఫిబ్రవరి పందొమ్మిదినే దాడికి సిద్ధపడ్డారు
Prediction: గరపంనరదడర
----------------------------------------------------------------------------------------------------
Target    : అయినా ఏ మాత్రం ప్రతిస్పందన రాలేదు
Prediction: కడికదసడనరలడ
----------------------------------------------------------------------------------------------------
Tar

Epoch 3/5
1/1 [==============================] - 0s 47ms/step
----------------------------------------------------------------------------------------------------
Average CER: 73.61%
----------------------------------------------------------------------------------------------------
Target    : పిన్ కోడ్ అయిదు వందల ముప్పై నాలుగు వందల అరవై ఆరు
Prediction: ంకదడనముదఅనవు
----------------------------------------------------------------------------------------------------
Target    : ఇక్కడ మామిడి తోటలు కూడా ఉన్నాయి
Prediction: కమడరికతలుడ
----------------------------------------------------------------------------------------------------
Target    : దీని పై మీ అభిప్రాయం ఏమిటి
Prediction: పమలడిమి
----------------------------------------------------------------------------------------------------
Target    : దీనిని పగటి వేషంగా భావించ వచ్చు
Prediction: దినదిశచక
----------------------------------------------------------------------------------------------------
Target    : పాండవులు మహాప్రస్థానం ప

Epoch 4/5
1/1 [==============================] - 0s 59ms/step
----------------------------------------------------------------------------------------------------
Average CER: 75.92%
----------------------------------------------------------------------------------------------------
Target    : అది కాపీహక్కుల ఉల్లంఘన కూడా కావచ్చును
Prediction: కరదకడకచ
----------------------------------------------------------------------------------------------------
Target    : మొఘల్ చక్రవర్తి షాజహాన్ కు నాయనమ్మ
Prediction: మకతజకనమక
----------------------------------------------------------------------------------------------------
Target    : నీవు ఈ జగములంతకూ శివుడవు
Prediction: ిదకడ
----------------------------------------------------------------------------------------------------
Target    : స్వామీజీ చికాగోకు తిరిగి వచ్చాడు
Prediction: మికతదితడ
----------------------------------------------------------------------------------------------------
Target    : ఇతని తండ్రి పేరు చుక్కపల్లి తిరుమలయ్య
Pred

Epoch 5/5
1/1 [==============================] - 0s 66ms/step
----------------------------------------------------------------------------------------------------
Average CER: 70.45%
----------------------------------------------------------------------------------------------------
Target    : అయినా ఏ మాత్రం ప్రతిస్పందన రాలేదు
Prediction: కనమదతిసపడనరలడు
----------------------------------------------------------------------------------------------------
Target    : కర్మజన్మ తాజా పుస్తకం
Prediction: కమచినతజతరతకం
----------------------------------------------------------------------------------------------------
Target    : అతని కుమార్తెను దొంగలు అపహరిస్తారు
Prediction: నుమకమబువలసత
----------------------------------------------------------------------------------------------------
Target    : దీని మొక్క తీగ జాతికి చెందినది
Prediction: దినిమకతిచతిదితినినది
----------------------------------------------------------------------------------------------------
Target    : వినాయకునికే ఆధిపత్యము

In [23]:
#callback function to check transcription on the val set
validation_callback = CallbackEval(validation_dataset)
#Train the model

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=1,
    callbacks=[validation_callback],
)

1/1 [==============================] - 0s 203ms/step
----------------------------------------------------------------------------------------------------
Average CER: 96.85%
----------------------------------------------------------------------------------------------------
Target    : ప్రజలకు సంబంధించిన సర్వీసులు అన్నీ వీటి ద్వారానే జరుగుతున్నాయి
Prediction:  ి ి
----------------------------------------------------------------------------------------------------
Target    : అయిదు వందల మీటర్లలో
Prediction:  
----------------------------------------------------------------------------------------------------
Target    : కృష్ణమాచారి విపుల వ్యాస పరంపర
Prediction:  ా
----------------------------------------------------------------------------------------------------
Target    : మిగతా వాళ్ళంతా జంతువుల్లాగా బాధ్యులుకారు
Prediction: ి ా
----------------------------------------------------------------------------------------------------
Target    : ఈ గ్రామ సర్పంచ్ మన్నెమానెమ్మ
Prediction:  
--

In [20]:
#inference
predictions= []
targets = []
for batch in validation_dataset:
  x, y =batch
  batch_predictions = model.predict(x)
  batch_predictions = decode_batch_predictions(batch_predictions)
  predictions.extend(batch_predictions)
  for label in y:
    label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
    targets.append(label)
avg_cer = calculate_cer_batch(predictions, targets)
print("-"* 100)
print("Average CER: {:.2%}".format(avg_cer))
print("-" * 100)
for i in np.random.randint(0, len(predictions), 5):
  print(f"Target    : {targets[i]}")
  print(f"Prediction: {predictions[i]}")
  print("-" * 100)
  

1/1 [==============================] - 0s 48ms/step
----------------------------------------------------------------------------------------------------
Average CER: 60.44%
----------------------------------------------------------------------------------------------------
Target    : కనుక అతడిని రక్షించుకోవడం నా కర్తవ్యం
Prediction: ఖనకాఅటడినక్షిచజుకవడకర్తవ
----------------------------------------------------------------------------------------------------
Target    : దీని ముందు డిడిటి చేసే హాని చాలా తక్కువ
Prediction: దీని ందుభింతఈీచిసనిలాతక
----------------------------------------------------------------------------------------------------
Target    : అది పగులి చెరుకు పాలు తెర్లతాయి
Prediction: అపగుకుతదిలత
----------------------------------------------------------------------------------------------------
Target    : ఇందులో పరతత్వం అనగా పరబ్రహ్మం
Prediction: ందురతనపరబడర
----------------------------------------------------------------------------------------------------
Target    : ఆ